# Process of Reference Value of HR46

In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_df_err(df, df_sub, ref=None):
    if ref is None:
        ref = df_sub
    # In case if column headers of reference is not the same to df
    if isinstance(df_sub, pd.DataFrame):
        df_sub.columns = df.columns
        ref.columns = df.columns
    df_err_val = df.sub(df_sub, axis="index")
    df_err_rel = df_err_val.div(ref, axis="index") * 100
    df_z = df - df.mean()
    ref_z = df_sub - df_sub.mean()
    df_err = {
        "MaxE/A^3": df_err_val.abs().max(),
        "MAD/A^3": df_err_val.abs().mean(),
        "RMSD/A^3": np.sqrt((df_err_val**2).mean()),
        "RelMaxE/%": df_err_rel.abs().max(),
        "RelMAD/%": df_err_rel.abs().mean(),
        "RelRMSD/%": np.sqrt((df_err_rel**2).mean()),
    }
    # In case df is pd.Series instead of pd.DataFrame 
    try:
        return pd.DataFrame(df_err).T
    except ValueError:
        return pd.Series(df_err)

## Read Data

In [3]:
df_cc_iso = pd.read_csv("HR46-CCSDt-isotropic.csv", header=[0], index_col=[0, 1])
df_cc_aniso = pd.read_csv("HR46-CCSDt-anisotropic.csv", header=[0], index_col=[0, 1])

In [4]:
df_mp_iso = pd.read_csv("HR46-RIMP2-isotropic.csv", header=[0], index_col=[0, 1])
df_mp_aniso = pd.read_csv("HR46-RIMP2-anisotropic.csv", header=[0], index_col=[0, 1])

In [5]:
mol_list = list(df_cc_iso.loc["aug-cc-pVDZ"].index)

In [6]:
mol_set1 = ['bromine', 'chlorine', 'nitrogen', 'oxygen', 'carbon_monoxide', 'sulfur_monoxide', 'carbon_dioxide', 'sulfur_dioxide', 'water', 'phosphine', 'hydrogen_sulfide', 'silane', 'ammonia', 'nitrogen_monoxide']
mol_set2 = ['ethene', 'propane', '1-3-butadiene', 'methanol', 'ethanol', 'methylformate', 'acetone', 'acetaldehyde', 'acetic_acid', 'dimethyl_ether', 'dimethyl_sulfide', 'fluoromethane', 'acetonitrile', 'pyrazole']
mol_set3 = ['isobutene', '1-pentene', 'methylacetate', 'N-methylacetamide', 'dimethyl_sulfone', 'dimethyl_amine', 'trimethyl_amine', 'imidazole', 'pyrrole', 'furan', 'thiophene', 'pyridine', 'phenol', 'chlorobenzene', 'fluorobenzene', 'toluene', 'benzene', 'cytosine']


In [7]:
print(len(mol_set1), len(mol_set2), len(mol_set3))

14 14 18


## Correlation Data

In [8]:
df_corr2_iso = df_mp_iso["RI-MP2"] - df_mp_iso["RI-JK"]
df_corr2_aniso = df_mp_aniso["RI-MP2"] - df_mp_aniso["RI-JK"]

In [9]:
df_corrd_iso = df_cc_iso["CCSD"] - df_cc_iso["MP2"]
df_corrd_aniso = df_cc_aniso["CCSD"] - df_cc_aniso["MP2"]

In [10]:
df_corrdt_iso = df_cc_iso["CCSD(T)"] - df_cc_iso["MP2"]
df_corrdt_aniso = df_cc_aniso["CCSD(T)"] - df_cc_aniso["MP2"]

## Benchmark

### Error between RI-JK/RI-Corr2 and numerical SCF/Corr2 (sec 3.1)

In [11]:
get_df_err(
    df_mp_iso.loc["aug-cc-pVTZ"]["RI-JK"],
    df_cc_iso.loc["aug-cc-pVTZ"]["SCF"])["RelRMSD/%"]

0.018311887969626597

In [12]:
get_df_err(
    df_mp_iso.loc["aug-cc-pVTZ"]["RI-MP2"] - df_mp_iso.loc["aug-cc-pVTZ"]["RI-JK"],
    df_cc_iso.loc["aug-cc-pVTZ"]["MP2"] - df_cc_iso.loc["aug-cc-pVTZ"]["SCF"],
    df_cc_iso.loc["aug-cc-pVTZ"]["MP2"]
)["RelRMSD/%"]

0.010148092563830638

## Isotropic Reference (Table S1)

In [13]:
df_ref_iso = pd.DataFrame(index=mol_list, columns=["SCF", "Corr2", "CorrD", "CorrD(T)", "MP2", "CCSD", "CCSD(T)"], dtype=float)
df_ref_iso.loc[:, "SCF"] = df_mp_iso.loc[("aug-cc-pCV5Z"), "RI-JK"]
df_ref_iso.loc[:, "Corr2"] = (125 * df_corr2_iso.loc["aug-cc-pCV5Z"] - 64 * df_corr2_iso.loc["aug-cc-pCVQZ"]) / 61

In [14]:
df_ref_iso.loc[mol_set1, "CorrD"] = (125 * df_corrd_iso.loc["aug-cc-pV5Z"].loc[mol_set1] - 64 * df_corrd_iso.loc["aug-cc-pVQZ"].loc[mol_set1]) / 61
df_ref_iso.loc[mol_set2, "CorrD"] = (64 * df_corrd_iso.loc["aug-cc-pVQZ"].loc[mol_set2] - 27 * df_corrd_iso.loc["aug-cc-pVTZ"].loc[mol_set2]) / 37
df_ref_iso.loc[mol_set3, "CorrD"] = (27 * df_corrd_iso.loc["aug-cc-pVTZ"].loc[mol_set3] - 8 * df_corrd_iso.loc["aug-cc-pVDZ"].loc[mol_set3]) / 19

In [15]:
df_ref_iso.loc[mol_set1, "CorrD(T)"] = (125 * df_corrdt_iso.loc["aug-cc-pV5Z"].loc[mol_set1] - 64 * df_corrdt_iso.loc["aug-cc-pVQZ"].loc[mol_set1]) / 61
df_ref_iso.loc[mol_set2, "CorrD(T)"] = (64 * df_corrdt_iso.loc["aug-cc-pVQZ"].loc[mol_set2] - 27 * df_corrdt_iso.loc["aug-cc-pVTZ"].loc[mol_set2]) / 37
df_ref_iso.loc[mol_set3, "CorrD(T)"] = (27 * df_corrdt_iso.loc["aug-cc-pVTZ"].loc[mol_set3] - 8 * df_corrdt_iso.loc["aug-cc-pVDZ"].loc[mol_set3]) / 19

In [16]:
df_ref_iso.loc[:, "MP2"] = df_ref_iso["SCF"] + df_ref_iso["Corr2"]
df_ref_iso.loc[:, "CCSD"] = df_ref_iso["MP2"] + df_ref_iso["CorrD"]
df_ref_iso.loc[:, "CCSD(T)"] = df_ref_iso["MP2"] + df_ref_iso["CorrD(T)"]

In [17]:
df_ref_iso.to_csv("HR46-ref-iso.csv")

## Anisotropic Reference (Table S2)

In [18]:
df_ref_aniso = pd.DataFrame(index=mol_list, columns=["SCF", "Corr2", "CorrD", "CorrD(T)", "MP2", "CCSD", "CCSD(T)"], dtype=float)
df_ref_aniso.loc[:, "SCF"] = df_mp_aniso.loc[("aug-cc-pCV5Z"), "RI-JK"]
df_ref_aniso.loc[:, "Corr2"] = (125 * df_corr2_aniso.loc["aug-cc-pCV5Z"] - 64 * df_corr2_aniso.loc["aug-cc-pCVQZ"]) / 61

In [19]:
df_ref_aniso.loc[mol_set1, "CorrD"] = (125 * df_corrd_aniso.loc["aug-cc-pV5Z"].loc[mol_set1] - 64 * df_corrd_aniso.loc["aug-cc-pVQZ"].loc[mol_set1]) / 61
df_ref_aniso.loc[mol_set2, "CorrD"] = (64 * df_corrd_aniso.loc["aug-cc-pVQZ"].loc[mol_set2] - 27 * df_corrd_aniso.loc["aug-cc-pVTZ"].loc[mol_set2]) / 37
df_ref_aniso.loc[mol_set3, "CorrD"] = (27 * df_corrd_aniso.loc["aug-cc-pVTZ"].loc[mol_set3] - 8 * df_corrd_aniso.loc["aug-cc-pVDZ"].loc[mol_set3]) / 19

In [20]:
df_ref_aniso.loc[mol_set1, "CorrD(T)"] = (125 * df_corrdt_aniso.loc["aug-cc-pV5Z"].loc[mol_set1] - 64 * df_corrdt_aniso.loc["aug-cc-pVQZ"].loc[mol_set1]) / 61
df_ref_aniso.loc[mol_set2, "CorrD(T)"] = (64 * df_corrdt_aniso.loc["aug-cc-pVQZ"].loc[mol_set2] - 27 * df_corrdt_aniso.loc["aug-cc-pVTZ"].loc[mol_set2]) / 37
df_ref_aniso.loc[mol_set3, "CorrD(T)"] = (27 * df_corrdt_aniso.loc["aug-cc-pVTZ"].loc[mol_set3] - 8 * df_corrdt_aniso.loc["aug-cc-pVDZ"].loc[mol_set3]) / 19

In [21]:
df_ref_aniso.loc[:, "MP2"] = df_ref_aniso["SCF"] + df_ref_aniso["Corr2"]
df_ref_aniso.loc[:, "CCSD"] = df_ref_aniso["MP2"] + df_ref_aniso["CorrD"]
df_ref_aniso.loc[:, "CCSD(T)"] = df_ref_aniso["MP2"] + df_ref_aniso["CorrD(T)"]

In [22]:
df_ref_aniso.to_csv("HR46-ref-aniso.csv")

## Benchmark

### Isotropic (Table 7)

In [23]:
get_df_err(df_ref_iso[["SCF", "MP2", "CCSD"]], df_ref_iso["CCSD(T)"])

,SCF,MP2,CCSD
MaxE/A^3,0.963407,0.212218,0.244982
MAD/A^3,0.185464,0.076586,0.068826
RMSD/A^3,0.263388,0.095156,0.081793
RelMaxE/%,10.172358,13.471855,2.361176
RelMAD/%,3.720478,1.575539,1.185491
RelRMSD/%,4.727390,2.609246,1.293964


In [24]:
get_df_err(
    df_ref_iso[["SCF", "MP2", "CCSD"]].drop(["oxygen", "sulfur_monoxide", "nitrogen_monoxide"]),
    df_ref_iso["CCSD(T)"].drop(["oxygen", "sulfur_monoxide", "nitrogen_monoxide"]))

,SCF,MP2,CCSD
MaxE/A^3,0.963407,0.204566,0.244982
MAD/A^3,0.192487,0.070207,0.072927
RMSD/A^3,0.271387,0.087107,0.084542
RelMaxE/%,10.172358,2.177740,2.361176
RelMAD/%,3.654022,1.096007,1.233097
RelRMSD/%,4.681686,1.185384,1.327843


### Anisotropic (Table 8)

In [25]:
mask = df_ref_aniso.index[df_ref_aniso["CCSD(T)"] > 0.5]

In [26]:
get_df_err(df_ref_aniso[["SCF", "MP2", "CCSD"]].loc[mask], df_ref_aniso["CCSD(T)"].loc[mask])

,SCF,MP2,CCSD
MaxE/A^3,1.304239,0.501903,0.208538
MAD/A^3,0.263022,0.070725,0.048371
RMSD/A^3,0.369269,0.132267,0.061511
RelMaxE/%,47.799659,46.415432,4.951142
RelMAD/%,10.793123,4.175164,2.014702
RelRMSD/%,13.901290,10.531904,2.371577


In [27]:
get_df_err(
    df_ref_aniso[["SCF", "MP2", "CCSD"]].loc[mask.drop(["oxygen", "sulfur_monoxide", "nitrogen_monoxide"])],
    df_ref_aniso["CCSD(T)"].loc[mask.drop(["oxygen", "sulfur_monoxide", "nitrogen_monoxide"])])

,SCF,MP2,CCSD
MaxE/A^3,1.304239,0.174439,0.208538
MAD/A^3,0.258041,0.043134,0.048836
RMSD/A^3,0.368518,0.066902,0.062488
RelMaxE/%,23.990183,9.322690,4.951142
RelMAD/%,9.422612,1.482850,1.883892
RelRMSD/%,11.144069,2.262999,2.219415


In [28]:
len(mask)

39

## Set Err

In [29]:
index_seterr_list = [
    "Set I, aV[DT]Z, aV[Q5]Z",
    "Set I, aV[TQ]Z, aV[Q5]Z",
    "Set II, aV[DT]Z, aV[TQ]Z"
]

### Isotropic (Table 5 and Table S5)

In [30]:
mol_set1 = ['bromine', 'chlorine', 'nitrogen', 'oxygen', 'carbon_monoxide', 'sulfur_monoxide', 'carbon_dioxide', 'sulfur_dioxide', 'water', 'phosphine', 'hydrogen_sulfide', 'silane', 'ammonia', 'nitrogen_monoxide']

In [31]:
df_iso_seterr = pd.DataFrame(index=index_seterr_list, columns=["CorrD", "CorrD(T)"])

In [32]:
df_iso_seterr.loc["Set I, aV[DT]Z, aV[Q5]Z", "CorrD"] = get_df_err(
    ( 27 * df_corrd_iso.loc["aug-cc-pVTZ"].loc[mol_set1] -  8 * df_corrd_iso.loc["aug-cc-pVDZ"].loc[mol_set1]) / 19,
    (125 * df_corrd_iso.loc["aug-cc-pV5Z"].loc[mol_set1] - 64 * df_corrd_iso.loc["aug-cc-pVQZ"].loc[mol_set1]) / 61,
    df_ref_iso.loc[mol_set1, "CCSD(T)"]
).loc["RelRMSD/%"]
df_iso_seterr.loc["Set I, aV[TQ]Z, aV[Q5]Z", "CorrD"] = get_df_err(
    ( 64 * df_corrd_iso.loc["aug-cc-pVQZ"].loc[mol_set1] - 27 * df_corrd_iso.loc["aug-cc-pVTZ"].loc[mol_set1]) / 37,
    (125 * df_corrd_iso.loc["aug-cc-pV5Z"].loc[mol_set1] - 64 * df_corrd_iso.loc["aug-cc-pVQZ"].loc[mol_set1]) / 61,
    df_ref_iso.loc[mol_set1, "CCSD(T)"]
).loc["RelRMSD/%"]
df_iso_seterr.loc["Set II, aV[DT]Z, aV[TQ]Z", "CorrD"] = get_df_err(
    ( 27 * df_corrd_iso.loc["aug-cc-pVTZ"].loc[mol_set2] -  8 * df_corrd_iso.loc["aug-cc-pVDZ"].loc[mol_set2]) / 19,
    ( 64 * df_corrd_iso.loc["aug-cc-pVQZ"].loc[mol_set2] - 27 * df_corrd_iso.loc["aug-cc-pVTZ"].loc[mol_set2]) / 37,
    df_ref_iso.loc[mol_set2, "CCSD(T)"]
).loc["RelRMSD/%"]

In [33]:
df_iso_seterr.loc["Set I, aV[DT]Z, aV[Q5]Z", "CorrD(T)"] = get_df_err(
    ( 27 * df_corrdt_iso.loc["aug-cc-pVTZ"].loc[mol_set1] -  8 * df_corrdt_iso.loc["aug-cc-pVDZ"].loc[mol_set1]) / 19,
    (125 * df_corrdt_iso.loc["aug-cc-pV5Z"].loc[mol_set1] - 64 * df_corrdt_iso.loc["aug-cc-pVQZ"].loc[mol_set1]) / 61,
    df_ref_iso.loc[mol_set1, "CCSD(T)"]
).loc["RelRMSD/%"]
df_iso_seterr.loc["Set I, aV[TQ]Z, aV[Q5]Z", "CorrD(T)"] = get_df_err(
    ( 64 * df_corrdt_iso.loc["aug-cc-pVQZ"].loc[mol_set1] - 27 * df_corrdt_iso.loc["aug-cc-pVTZ"].loc[mol_set1]) / 37,
    (125 * df_corrdt_iso.loc["aug-cc-pV5Z"].loc[mol_set1] - 64 * df_corrdt_iso.loc["aug-cc-pVQZ"].loc[mol_set1]) / 61,
    df_ref_iso.loc[mol_set1, "CCSD(T)"]
).loc["RelRMSD/%"]
df_iso_seterr.loc["Set II, aV[DT]Z, aV[TQ]Z", "CorrD(T)"] = get_df_err(
    ( 27 * df_corrdt_iso.loc["aug-cc-pVTZ"].loc[mol_set2] -  8 * df_corrdt_iso.loc["aug-cc-pVDZ"].loc[mol_set2]) / 19,
    ( 64 * df_corrdt_iso.loc["aug-cc-pVQZ"].loc[mol_set2] - 27 * df_corrdt_iso.loc["aug-cc-pVTZ"].loc[mol_set2]) / 37,
    df_ref_iso.loc[mol_set2, "CCSD(T)"]
).loc["RelRMSD/%"]

In [34]:
df_iso_seterr

,CorrD,CorrD(T)
"Set I, aV[DT]Z, aV[Q5]Z",0.162937,0.157771
"Set I, aV[TQ]Z, aV[Q5]Z",0.150835,0.12738
"Set II, aV[DT]Z, aV[TQ]Z",0.084144,0.09087


### Anisotropic (Table 6 and Table S6)

In [35]:
mol_set1 = ['bromine', 'chlorine', 'nitrogen', 'carbon_monoxide', 'carbon_dioxide', 'sulfur_dioxide', 'water', 'phosphine', 'hydrogen_sulfide', 'silane', 'ammonia']

In [36]:
df_aniso_seterr = pd.DataFrame(index=index_seterr_list, columns=["CorrD", "CorrD(T)"])

In [37]:
mol_set1_aniso = list(df_ref_aniso.loc[mol_set1, "CCSD(T)"].index[df_ref_aniso.loc[mol_set1, "CCSD(T)"] > 0.5])
mol_set2_aniso = list(df_ref_aniso.loc[mol_set2, "CCSD(T)"].index[df_ref_aniso.loc[mol_set2, "CCSD(T)"] > 0.5])
print(mol_set1_aniso)
print(mol_set2_aniso)

['bromine', 'chlorine', 'nitrogen', 'carbon_monoxide', 'carbon_dioxide', 'sulfur_dioxide']
['ethene', 'propane', '1-3-butadiene', 'ethanol', 'methylformate', 'acetone', 'acetaldehyde', 'acetic_acid', 'dimethyl_ether', 'dimethyl_sulfide', 'acetonitrile', 'pyrazole']


In [38]:
print(len(mol_set1_aniso))
print(len(mol_set2_aniso))

6
12


In [39]:
df_aniso_seterr.loc["Set I, aV[DT]Z, aV[Q5]Z", "CorrD"] = get_df_err(
    ( 27 * df_corrd_aniso.loc["aug-cc-pVTZ"].loc[mol_set1_aniso] -  8 * df_corrd_aniso.loc["aug-cc-pVDZ"].loc[mol_set1_aniso]) / 19,
    (125 * df_corrd_aniso.loc["aug-cc-pV5Z"].loc[mol_set1_aniso] - 64 * df_corrd_aniso.loc["aug-cc-pVQZ"].loc[mol_set1_aniso]) / 61,
    df_ref_aniso.loc[mol_set1, "CCSD(T)"]
).loc["RelRMSD/%"]
df_aniso_seterr.loc["Set I, aV[TQ]Z, aV[Q5]Z", "CorrD"] = get_df_err(
    ( 64 * df_corrd_aniso.loc["aug-cc-pVQZ"].loc[mol_set1_aniso] - 27 * df_corrd_aniso.loc["aug-cc-pVTZ"].loc[mol_set1_aniso]) / 37,
    (125 * df_corrd_aniso.loc["aug-cc-pV5Z"].loc[mol_set1_aniso] - 64 * df_corrd_aniso.loc["aug-cc-pVQZ"].loc[mol_set1_aniso]) / 61,
    df_ref_aniso.loc[mol_set1, "CCSD(T)"]
).loc["RelRMSD/%"]
df_aniso_seterr.loc["Set II, aV[DT]Z, aV[TQ]Z", "CorrD"] = get_df_err(
    ( 27 * df_corrd_aniso.loc["aug-cc-pVTZ"].loc[mol_set2_aniso] -  8 * df_corrd_aniso.loc["aug-cc-pVDZ"].loc[mol_set2_aniso]) / 19,
    ( 64 * df_corrd_aniso.loc["aug-cc-pVQZ"].loc[mol_set2_aniso] - 27 * df_corrd_aniso.loc["aug-cc-pVTZ"].loc[mol_set2_aniso]) / 37,
    df_ref_aniso.loc[mol_set2, "CCSD(T)"]
).loc["RelRMSD/%"]

In [40]:
df_aniso_seterr.loc["Set I, aV[DT]Z, aV[Q5]Z", "CorrD(T)"] = get_df_err(
    ( 27 * df_corrdt_aniso.loc["aug-cc-pVTZ"].loc[mol_set1_aniso] -  8 * df_corrdt_aniso.loc["aug-cc-pVDZ"].loc[mol_set1_aniso]) / 19,
    (125 * df_corrdt_aniso.loc["aug-cc-pV5Z"].loc[mol_set1_aniso] - 64 * df_corrdt_aniso.loc["aug-cc-pVQZ"].loc[mol_set1_aniso]) / 61,
    df_ref_aniso.loc[mol_set1, "CCSD(T)"]
).loc["RelRMSD/%"]
df_aniso_seterr.loc["Set I, aV[TQ]Z, aV[Q5]Z", "CorrD(T)"] = get_df_err(
    ( 64 * df_corrdt_aniso.loc["aug-cc-pVQZ"].loc[mol_set1_aniso] - 27 * df_corrdt_aniso.loc["aug-cc-pVTZ"].loc[mol_set1_aniso]) / 37,
    (125 * df_corrdt_aniso.loc["aug-cc-pV5Z"].loc[mol_set1_aniso] - 64 * df_corrdt_aniso.loc["aug-cc-pVQZ"].loc[mol_set1_aniso]) / 61,
    df_ref_aniso.loc[mol_set1, "CCSD(T)"]
).loc["RelRMSD/%"]
df_aniso_seterr.loc["Set II, aV[DT]Z, aV[TQ]Z", "CorrD(T)"] = get_df_err(
    ( 27 * df_corrdt_aniso.loc["aug-cc-pVTZ"].loc[mol_set2_aniso] -  8 * df_corrdt_aniso.loc["aug-cc-pVDZ"].loc[mol_set2_aniso]) / 19,
    ( 64 * df_corrdt_aniso.loc["aug-cc-pVQZ"].loc[mol_set2_aniso] - 27 * df_corrdt_aniso.loc["aug-cc-pVTZ"].loc[mol_set2_aniso]) / 37,
    df_ref_aniso.loc[mol_set2, "CCSD(T)"]
).loc["RelRMSD/%"]

In [41]:
df_aniso_seterr

,CorrD,CorrD(T)
"Set I, aV[DT]Z, aV[Q5]Z",0.299293,0.197042
"Set I, aV[TQ]Z, aV[Q5]Z",0.334538,0.276591
"Set II, aV[DT]Z, aV[TQ]Z",0.208453,0.224558


## Species Error

In [42]:
(df_ref_iso.loc["oxygen", "MP2"] / df_ref_iso.loc["oxygen", "CCSD(T)"] - 1) * 100

-13.471854937174788

In [43]:
(df_ref_iso.loc["nitrogen_monoxide", "MP2"] / df_ref_iso.loc["nitrogen_monoxide", "CCSD(T)"] - 1) * 100

-6.553919686161624

In [44]:
(df_ref_iso.loc["sulfur_monoxide", "MP2"] / df_ref_iso.loc["sulfur_monoxide", "CCSD(T)"] - 1) * 100

-5.320714636207303

In [45]:
(df_ref_aniso.loc["oxygen", "MP2"] / df_ref_aniso.loc["oxygen", "CCSD(T)"] - 1) * 100

-46.41543155772775

In [46]:
(df_ref_aniso.loc["nitrogen_monoxide", "MP2"] / df_ref_aniso.loc["nitrogen_monoxide", "CCSD(T)"] - 1) * 100

-32.046388011331686

In [47]:
(df_ref_aniso.loc["sulfur_monoxide", "MP2"] / df_ref_aniso.loc["sulfur_monoxide", "CCSD(T)"] - 1) * 100

-30.986991522531426